In [ ]:
!pip install tensorflow==2.5
import tensorflow as tf
print(tf.__version__)

     |████████████████████████████████| 454.3 MB 15 kB/s 
     |████████████████████████████████| 1.2 MB 30.2 MB/s 
     |████████████████████████████████| 4.0 MB 35.0 MB/s 
     |████████████████████████████████| 462 kB 53.6 MB/s 
  Created wheel for wrapt: filename=wrapt-1.12.1-cp37-cp37m-linux_x86_64.whl size=68719 sha256=c7248f731cb4d845e7822212d6e5eb23a52e1f1d6a69ea8476cfb0ed2d554ce2
  Stored in directory: /root/.cache/pip/wheels/62/76/4c/aa25851149f3f6d9785f6c869387ad82b3fd37582fa8147ac6
Successfully built wrapt
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Uninstalling typing-extensions-3.10.0.2:
      Successfully uninstalled typing-extensions-3.10.0.2
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.42.0
    Uninstalling grpcio-1.42.0:
      Successfully uninstalled grpcio-1.42.0
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.13.3
    Uninstalling wrapt-1.13.3:
      Suc

In [ ]:
print(tf.__version__)

2.5.0


In [ ]:
#Basic libraries
import pandas as pd 
import numpy as np 

#Visualization libraries
import matplotlib.pyplot as plt 
from matplotlib import rcParams
import seaborn as sns
from textblob import TextBlob
from plotly import tools
import plotly.graph_objs as go
from plotly.offline import iplot
%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 5]
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

#NLTK libraries
import nltk
import re
import string
from nltk.corpus import stopwords
from wordcloud import WordCloud,STOPWORDS
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
# Machine Learning libraries
import sklearn 
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression

from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
 

#Metrics libraries
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

#Miscellanous libraries
from collections import Counter

#Ignore warnings
import warnings
warnings.filterwarnings('ignore')

#Deep learning libraries
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

In [ ]:
import pandas as pd
df = pd.read_csv('AbstractDataset_trainSet_FULL.csv')
df.head()

,id,file_name,text,#label
0,1,2111.09381_originalAbstract.txt,"We present MEDCOD, a Medically- Accurate, Emot...",1
1,2,2111.09381_generatedAbstract.txt,"We present MEDCOD, a Medically- Accurate, Emot...",0
2,3,2111.09388_originalAbstract.txt,This work applies Minimum Bayes Risk (MBR) dec...,1
3,4,2111.09388_generatedAbstract.txt,This work applies Minimum Bayes Risk (MBR) dec...,0
4,5,2111.09412_originalAbstract.txt,"In this study, we present a meta-learning mode...",1


In [ ]:
#get the independent features
X = df['text']
#X.head()
#get dependent variables
y = df['#label']
#y.head()

In [ ]:
df.shape

(200, 4)

In [ ]:
#Checking dataset for any missing values and those are dropped
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer,HashingVectorizer
df = df.dropna()
#df.head(10)

In [ ]:
#create a copy of this dataset and also reset its index values.
messages = df.copy()
messages.reset_index(inplace = True)
#messages.head(10)

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
#Stopword removal and Stemming
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['text'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)
corpus[3]

'work appli minimum bay risk mbr decod optim divers auto mate metric translat qualiti au tomat metric machin translat made tremend progress recent par ticular neural metric fine tune human rate e g bleurt comet perform surfac metric term cor relat human judgement experi ment show model learn approxim human defin represent low perform improv obtain translat differ classic beam search output translat much lower likelihood less favor sur face metric like bleu'

In [ ]:
df.shape

(200, 4)

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

In [ ]:

#Setting up vocabulary size
voc_size=10000

#One hot encoding 
onehot_repr=[one_hot(words,voc_size)for words in corpus]

In [ ]:
df.shape

(200, 4)

In [ ]:
#Setting sentence length
sent_length=5000

#Padding the sentences
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 1643 8343 9435]
 [   0    0    0 ... 1643 8343 9435]
 [   0    0    0 ... 6563 4075 8138]
 ...
 [   0    0    0 ... 8762 8861 9659]
 [   0    0    0 ... 7841  863 9318]
 [   0    0    0 ... 7841  863 9318]]


In [ ]:
df.shape

(200, 4)

In [ ]:
#Creating the lstm model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100)) #Adding 100 lstm neurons in the layer
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))

#Compiling the model
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 5000, 40)          400000    
_________________________________________________________________
dropout (Dropout)            (None, 5000, 40)          0         
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 456,501
Trainable params: 456,501
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
df.shape

(200, 4)

In [ ]:
import numpy as np 
# Converting the X and y as array
X_final=np.array(embedded_docs)
y_final=np.array(y)

#Check shape of X and y final
X_final.shape,y_final.shape

((200, 5000), (200,))

In [ ]:
df.shape

(200, 4)

In [ ]:
from sklearn.model_selection import train_test_split
# Train test split of the X and y final
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

# Fitting with 10 epochs and 64 batch size
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
3/3 [==============================] - 25s 7s/step - loss: 0.6945 - accuracy: 0.4478 - val_loss: 0.6944 - val_accuracy: 0.3485
Epoch 2/10
3/3 [==============================] - 23s 7s/step - loss: 0.6921 - accuracy: 0.6045 - val_loss: 0.6956 - val_accuracy: 0.3030
Epoch 3/10
3/3 [==============================] - 22s 7s/step - loss: 0.6909 - accuracy: 0.6194 - val_loss: 0.6969 - val_accuracy: 0.3333
Epoch 4/10
3/3 [==============================] - 22s 7s/step - loss: 0.6900 - accuracy: 0.6567 - val_loss: 0.6987 - val_accuracy: 0.3485
Epoch 5/10
3/3 [==============================] - 22s 7s/step - loss: 0.6882 - accuracy: 0.6269 - val_loss: 0.7019 - val_accuracy: 0.3939
Epoch 6/10
3/3 [==============================] - 24s 8s/step - loss: 0.6845 - accuracy: 0.5821 - val_loss: 0.7076 - val_accuracy: 0.4545
Epoch 7/10
3/3 [==============================] - 22s 7s/step - loss: 0.6800 - accuracy: 0.5373 - val_loss: 0.7231 - val_accuracy: 0.4697
Epoch 8/10
3/3 [==================

In [ ]:
# Predicting from test data
y_pred=model.predict_classes(X_test)

#Creating confusion matrix
#confusion_matrix(y_test,y_pred)
cm = metrics.confusion_matrix(y_test, y_pred)
plot_confusion_matrix(cm,classes=['Fake','True'])

NameError: ignored

In [ ]:
#Checking for accuracy
accuracy_score(y_test,y_pred)

0.24242424242424243